<a href="https://colab.research.google.com/github/margual56/InteligenciaNegocioClasificadores/blob/main/Benchmark_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imprime el tiempo que tarda en ejecutar cada celda
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 14.2 MB/s 
time: 381 µs (started: 2022-11-17 20:34:17 +00:00)


In [2]:
# Instala el clasificador y open
!pip install tabpfn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 153 kB 15.6 MB/s 


In [3]:
import sys
import numpy as np
from pathlib import Path
import pandas as pd
import torch
import os
import time
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from pathlib import Path
import tensorflow as tf

from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier
from tabpfn.scripts.decision_boundary import DecisionBoundaryDisplay

In [ ]:
tf.compat.v1.enable_eager_execution()

class MyLayer(tf.keras.Model):
    def __init__(self, X_train, y_train, N_ensemble_configurations=16):
        super(MyLayer, self).__init__()
        self.block_1 = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.block_2 = tf.keras.layers.Dense(50, activation='relu')
        self.classifier = TabPFNClassifier(device='cuda', N_ensemble_configurations=N_ensemble_configurations)
        self.initial_fit(X_train, y_train)

    def initial_fit(self, X_train, y_train):
        x = self.block_1(X_train)
        x = self.block_2(x)
        self.classifier.fit(x, y_train, overwrite_warning=True)

    def call(self, inputs):
        x = self.block_1(inputs)
        x = self.block_2(x)
        return self.classifier.predict(tf.make_ndarray(x))

time: 2.06 ms (started: 2022-11-17 22:06:41 +00:00)


In [6]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

print(train_images.shape)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images = train_images / 255.0
test_images = test_images / 255.0


(60000, 28, 28)
[9 0 0 ... 3 0 5]
[0 1 1 ... 2 1 5]


In [ ]:
# tab = TabPFNClassifier(device='cuda', N_ensemble_configurations=16)
# print(train_images.shape)
# tab.fit(train_images, train_labels_num, overwrite_warning=True)

model = MyLayer(train_images, train_labels)

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
time: 415 ms (started: 2022-11-17 22:06:44 +00:00)


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

time: 7.99 ms (started: 2022-11-17 22:06:46 +00:00)


In [ ]:
model.fit(train_images, train_labels, epochs=10)

Epoch 1/10


TypeError: ignored

time: 453 ms (started: 2022-11-17 22:06:47 +00:00)


In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)